In [5]:
import json 
from collections import Counter
from tqdm import tqdm 
from colorama import Style,Fore

In [1]:
import importlib
import utils  # or whatever your module is called
importlib.reload(utils)

2025-04-18 06:45:32,309 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-MISC, B-MISC, E-MISC, I-MISC, S-ORG, B-ORG, E-ORG, I-ORG, <START>, <STOP>
2025-04-18 06:45:50,331 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-MISC, B-MISC, E-MISC, I-MISC, S-ORG, B-ORG, E-ORG, I-ORG, <START>, <STOP>


<module 'utils' from 'c:\\Users\\hp\\AgroTube\\filtering\\scripts\\utils.py'>

## Refining 1

- Refine the language field of the videos

In [ ]:
from utils import RefineLanguage1


In [ ]:
RefineLanguage1()

In [8]:
with open("../../collecting/jsons/videos.json", "r", encoding="utf-8") as file:
    videos = json.load(file)
len(videos)

6859

In [6]:
with open("../jsons/videosR1.json", "r", encoding="utf-8") as file:
    videosR1 = json.load(file)
len(videosR1)

6859

In [9]:
langue_counter = Counter(video['langue'] for video in videosR1)
print("Toutes les valeurs uniques de langue avec leur nombre d'occurrences :")
for langue, count in langue_counter.items():
    print(f"- {langue} : {count}")


Toutes les valeurs uniques de langue avec leur nombre d'occurrences :
-  : 2449
- fr : 3229
- fr-FR : 477
- fr-CA : 143
- zxx : 11
- en : 255
- pt : 1
- fr-CH : 24
- en-US : 138
- en-GB : 38
- fr-BE : 43
- hi : 6
- de : 3
- en-IN : 5
- ro : 1
- ur : 2
- de-CH : 2
- en-CA : 16
- fil : 1
- br : 1
- ar : 3
- nl : 1
- ru : 1
- es-419 : 2
- eu : 1
- ko : 4
- zh-CN : 1
- de-DE : 1


## Refining 2

- Add the Localisation Information

In [ ]:
from utils import RefineLanguage2

In [ ]:
RefineLanguage2()

In [ ]:
with open("../jsons/videosR2.json", "r", encoding="utf-8") as file:
    videosR2 = json.load(file)
len(videosR2)

In [ ]:
langue_counter = Counter(video['langue'] for video in videosR2)
print("Toutes les valeurs uniques de langue avec leur nombre d'occurrences :")
for langue, count in sorted(langue_counter.items(), key=lambda x: x[1], reverse=True):
    print(f"- {langue} : {count}")


In [ ]:
total = len(videosR2)

count_fr = sum(1 for video in videosR2 if video['langue'] == 'fr')
count_frFR = sum(1 for video in videosR2 if video['langue'] == 'fr-FR')
count_fr_autre = sum(1 for video in videosR2 if video['langue'].startswith('fr') and video['langue'] not in ['fr', 'fr-FR'])
count_autre = total - (count_fr + count_frFR + count_fr_autre)

print(f"Total de vidéos : {total}")
print(f"- fr       : {count_fr} ({count_fr / total:.2%})")
print(f"- fr-FR    : {count_frFR} ({count_frFR / total:.2%})")
print(f"- fr-autre : {count_fr_autre} ({count_fr_autre / total:.2%})")
print(f"- autre    : {count_autre} ({count_autre / total:.2%})")

In [ ]:
1332+4312+632

## Refining 3

- Get the localisation information for 'fr' videos :
  - Try to explore the video description+Title and Bio of the corresponding channel.

### 1.Seperate These videos 

In [ ]:
with open("../jsons/videosR2.json", "r", encoding="utf-8") as file:
    videosR2 = json.load(file)
videosfr = []
for video in videosR2:
    if video['langue']=='fr':
        videosfr.append(video)
with open("../jsons/videosfr.json", "w", encoding="utf-8") as f:
    json.dump(videosfr, f, ensure_ascii=False, indent=2)
    print(Style.BRIGHT+Fore.GREEN+'\n json saved'+Style.RESET_ALL)
    print(Style.BRIGHT+Fore.YELLOW+f'\n json length is : {len(videosfr)}'+Style.RESET_ALL)

### 2. Get the location by using metadata:

In [ ]:
# Get the Context first
import importlib
import utils  # or whatever your module is called
importlib.reload(utils)

from utils import getContextAll


In [ ]:
getContextAll()

### 3.NER

- with spacy (not realy accurate)

In [ ]:
import spacy

nlp = spacy.load("fr_core_news_lg")

In [ ]:
context = """
Le bio de la chaine : \n \n        \n###############\n        \nDescription : Hello ! Aujourd’hui Bruno nous invite à questionner l'usage que faisons de notre habitat et nous propose de réduire nos surfaces de vie tout en gardant notre confort, de construire notre maison sans faire de prêt bancaire sur 20 ans. Et si on voyait dans le fait de construire notre maison plus qu'un défi, un cheminement, une expérience humaine, un accomplissement ?\nDécouvrez la construction d’un habitat autonome avec une structure bois, isolation paille, enduit terre de 24m2 avec cuisine, séjour, chambre et salle de bain pour 5000€ ! \nPrenez bien soin de vous.  \nJe vous embrasse, \n\nA bientôt 🌸 \n\nSarah \n\n\n\nLe site internet d’Alter’éco 30 : https://bit.ly/37pw1uj \n\n#OurInnerTravels #Autonomie #Habitat #Maison #HabitatAutonome #Ecologie \n\n\nRetrouve moi aussi sur : \n🎙PODCAST : https://anchor.fm/our-inner-travels\n\n📸 I N S T A G R A M : https://instagram.com/ourinnertravels \n✍️ B L O G : https://ourinnertravels.com \n📸 FACEBOOK : https://www.facebook.com/OurInnerTravels \n💕 Abonne-toi à la chaine : https://bit.ly/2PUxikr \n💌 Me Contacter 💌: contact@ourinnertravels.com\n        \n        \n###############\n        \nDescription : Hello, Aujourd’hui je vous retrouve dans la première partie d’une série de 5 entretiens avec Bruno sur le thème : L’écologie et l’alternativité comme invitation à trouver un sens à la vie.\nPartie 1 : Echovert, la construction d’un lieu alternatif, autonome et écologique\nPartie 2 : Vivre de façon alternative, les joies\nPartie 3 : Vivre de façon alternative, les défis \nPartie 4 : L’écologie, une invitation à trouver un sens à la vie\nPartie 5 : Les 3 kiffs de Bruno\n\nVous préférez écouter la vidéo sous format podcast ? C'est ici : https://bit.ly/3gmy5ZF\n\nEn octobre, de retour de voyage la réadaptation à la vie parisienne est difficile, est-ce peut-être car je n’ai pas envie de me réadapter, que je n’y vois plus de sens ? Je pars donc en Camargue, à une demi-heure de Nîmes et passe un mois dans un éco-lieu à la recherche de sens, de rencontres, de personnes qui partagent mes idées et mes aspirations. Bruno et Anne ont créé ce lieu de vie alternatif où naissent inventions respectueuses de l’environnement et expérimentations d’outils pour l’autonomie alimentaire et énergétique. Avec Bruno, on a commencé par parler de ce lieu de vie alternatif et de ce qu’implique le fait de vivre en communauté. On s’est aussi demander comment se reconnecter à soi et aux autres dans un monde qui semble parfois déconnecté du réel. Au fil de nos paroles, on en est finalement venu à se questionner sur le sens de la vie, et en quoi le fait de cultiver son écologie intérieure est peut-être la première forme d’engagement écologique à avoir. Aujourd’hui je vous propose de partir en voyage intérieur grâce à Bruno. L’écologie et l’alternativité comme invitation à trouver un sens à la vie.\n\nA bientôt 🌸 \n\nSarah \n\nLe site internet d’Alter’éco 30 : https://bit.ly/37pw1uj \n\nRetrouve moi aussi sur : \n🎙PODCAST : https://anchor.fm/our-inner-travels\n📸 I N S T A G R A M : https://instagram.com/ourinnertravels \n✍️ B L O G : https://ourinnertravels.com \n📸 FACEBOOK : https://www.facebook.com/OurInnerTravels \n💕 Abonne-toi à la chaine : https://bit.ly/2PUxikr \n💌 Me Contacter 💌: contact@ourinnertravels.com\n        \n        \n###############\n\n        titre : Habitat autonome et autoconstruction\n        \n        \n###############\n\n        titre : PARLONS #2 - 1 : Echovert, la construction d’un lieu alternatif, autonome et écologique - Bruno\n       

"""

print(context)

In [ ]:
doc = nlp(context)

for ent in doc.ents:
    if ent.label_ == "LOC":
        print(f"{ent}  :  {ent.label_}" )

In [2]:
from utils import RefineLanguage3

In [3]:
RefineLanguage3()

  0%|          | 1/1332 [00:00<12:45,  1.74it/s]


--unknown--

{}


  0%|          | 2/1332 [00:14<3:02:13,  8.22s/it]


--FR--

{
  "Vosges du Nord": "FR",
  "Région": "CN",
  "Parc naturel régional des Vosges": "FR",
  "Neuwiller-lès-Saverne": "FR"
}


  0%|          | 3/1332 [00:33<4:54:13, 13.28s/it]

...


--CA--

{
  "Auvergne": "CA",
  "la France": "US"
}


  8%|▊         | 108/1332 [52:57<5:53:43, 17.34s/it]


--unknown--

{}


  8%|▊         | 108/1332 [53:48<10:09:55, 29.90s/it]


KeyboardInterrupt: 

In [ ]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
login(os.getenv("HUGG_TOKEN"))


In [ ]:
from tqdm import tqdm
from flair.models import SequenceTagger
from flair.data import Sentence

# Enable tqdm progress bar explicitly
tagger = SequenceTagger.load("flair/ner-french")


In [ ]:
text = """
Le bio de la chaine : \n \n        \n###############\n        \nDescription : Hello ! Aujourd’hui Bruno nous invite à questionner l'usage que faisons de notre habitat et nous propose de réduire nos surfaces de vie tout en gardant notre confort, de construire notre maison sans faire de prêt bancaire sur 20 ans. Et si on voyait dans le fait de construire notre maison plus qu'un défi, un cheminement, une expérience humaine, un accomplissement ?\nDécouvrez la construction d’un habitat autonome avec une structure bois, isolation paille, enduit terre de 24m2 avec cuisine, séjour, chambre et salle de bain pour 5000€ ! \nPrenez bien soin de vous.  \nJe vous embrasse, \n\nA bientôt 🌸 \n\nSarah \n\n\n\nLe site internet d’Alter’éco 30 : https://bit.ly/37pw1uj \n\n#OurInnerTravels #Autonomie #Habitat #Maison #HabitatAutonome #Ecologie \n\n\nRetrouve moi aussi sur : \n🎙PODCAST : https://anchor.fm/our-inner-travels\n\n📸 I N S T A G R A M : https://instagram.com/ourinnertravels \n✍️ B L O G : https://ourinnertravels.com \n📸 FACEBOOK : https://www.facebook.com/OurInnerTravels \n💕 Abonne-toi à la chaine : https://bit.ly/2PUxikr \n💌 Me Contacter 💌: contact@ourinnertravels.com\n        \n        \n###############\n        \nDescription : Hello, Aujourd’hui je vous retrouve dans la première partie d’une série de 5 entretiens avec Bruno sur le thème : L’écologie et l’alternativité comme invitation à trouver un sens à la vie.\nPartie 1 : Echovert, la construction d’un lieu alternatif, autonome et écologique\nPartie 2 : Vivre de façon alternative, les joies\nPartie 3 : Vivre de façon alternative, les défis \nPartie 4 : L’écologie, une invitation à trouver un sens à la vie\nPartie 5 : Les 3 kiffs de Bruno\n\nVous préférez écouter la vidéo sous format podcast ? C'est ici : https://bit.ly/3gmy5ZF\n\nEn octobre, de retour de voyage la réadaptation à la vie parisienne est difficile, est-ce peut-être car je n’ai pas envie de me réadapter, que je n’y vois plus de sens ? Je pars donc en Camargue, à une demi-heure de Nîmes et passe un mois dans un éco-lieu à la recherche de sens, de rencontres, de personnes qui partagent mes idées et mes aspirations. Bruno et Anne ont créé ce lieu de vie alternatif où naissent inventions respectueuses de l’environnement et expérimentations d’outils pour l’autonomie alimentaire et énergétique. Avec Bruno, on a commencé par parler de ce lieu de vie alternatif et de ce qu’implique le fait de vivre en communauté. On s’est aussi demander comment se reconnecter à soi et aux autres dans un monde qui semble parfois déconnecté du réel. Au fil de nos paroles, on en est finalement venu à se questionner sur le sens de la vie, et en quoi le fait de cultiver son écologie intérieure est peut-être la première forme d’engagement écologique à avoir. Aujourd’hui je vous propose de partir en voyage intérieur grâce à Bruno. L’écologie et l’alternativité comme invitation à trouver un sens à la vie.\n\nA bientôt 🌸 \n\nSarah \n\nLe site internet d’Alter’éco 30 : https://bit.ly/37pw1uj \n\nRetrouve moi aussi sur : \n🎙PODCAST : https://anchor.fm/our-inner-travels\n📸 I N S T A G R A M : https://instagram.com/ourinnertravels \n✍️ B L O G : https://ourinnertravels.com \n📸 FACEBOOK : https://www.facebook.com/OurInnerTravels \n💕 Abonne-toi à la chaine : https://bit.ly/2PUxikr \n💌 Me Contacter 💌: contact@ourinnertravels.com\n        \n        \n###############\n\n        titre : Habitat autonome et autoconstruction\n        \n        \n###############\n\n        titre : PARLONS #2 - 1 : Echovert, la construction d’un lieu alternatif, autonome et écologique - Bruno\n       
"""
sentence = Sentence(text)
tagger.predict(sentence)

for entity in sentence.get_spans('ner'):
    label = entity.get_label("ner")
    if label.value == "LOC" and label.score >= 0.7:
        entity_text = entity.text
        entity_label = label.value
        entity_score = label.score
        print("Text:", entity_text)
        print("Label:", entity_label)
        print("Score:", entity_score)

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapi")
location = geolocator.geocode("FAcebook", language='fr', addressdetails=True, timeout=10)
location

## Filtering 1

- Remove fr-autre,autre
- keep only fr and fr-FR , a big amount of fr are for france.
- obtain the localisation of a channel if it's not providled by youtube api can be complexe because you can't find it anywhere.